This notebook does the computations needed during the audit. It can:
1. Compute the round size that achieves a desired stopping probability.
2. Compute the risk for a given sample (ie. compute the stopping condition).

This notebook uses the 'r2b2' open source ballot-polling audit software library:
https://github.com/gwexploratoryaudits/r2b2

In [59]:
# Imports
from r2b2.contest import ContestType, Contest
from r2b2.minerva2 import Minerva2
from r2b2.minerva import Minerva
from r2b2.eor_bravo import EOR_BRAVO
from r2b2.so_bravo import SO_BRAVO
import pandas as pd
import numpy as np

In [60]:
# Contest: School Construction and Renovation Projects
# Link to results: https://elections.ri.gov/publications/Election_Publications/Election_Info/LocalElections/11-2-21%20Portsmouth.pdf
contest_name = "\nSchool Construction and Renovation Projects"
tally = {'Approve' : 2391, 'Reject' : 1414}

Now we set the audit parameters.

In [61]:
# Set the desired stopping probability for the first round
sprob = .9

# Set the risk-limit
risk_limit = .1

# Some useful constants
reported_winner = max(tally, key=tally.get) 
winner_votes = tally[reported_winner]
total_relevant = sum(tally.values())
loser_votes = total_relevant - winner_votes
margin = (winner_votes / total_relevant) - (loser_votes / total_relevant)

# Make the contest object
contest_reported = Contest(total_relevant, 
                            tally, 
                            num_winners=1, 
                            reported_winners=[reported_winner],
                            contest_type=ContestType.PLURALITY)

# Make the audit objects. While we are running a Minerva 2.0 audit, we might as well compute other ballot-polling RLA numbers...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
minerva2 = Minerva2(risk_limit, 1.0, contest_reported)
minerva = Minerva(risk_limit, 1.0, contest_reported)
so_bravo = SO_BRAVO(risk_limit, 1.0, contest_reported)
eor_bravo = EOR_BRAVO(risk_limit, 1.0, contest_reported)

# Print out the audit parameters
print(contest_name)
print("Tally: {}".format(tally))
print("Total Ballots: {}".format(sum(tally.values())))
print("Margin: {}".format(margin))
print("Stopping Probability: {}".format(sprob))
print("Risk Limit: {}".format(risk_limit))


School Construction and Renovation Projects
Tally: {'Approve': 2391, 'Reject': 1414}
Total Ballots: 3805
Margin: 0.2567674113009198
Stopping Probability: 0.9
Risk Limit: 0.1


# First round size
Now let's compute the first round size!

In [62]:
minerva2_data = (minerva2, "Minerva 2.0", minerva2.next_sample_size(sprob))
minerva_data = (minerva, "Minerva", minerva.next_sample_size(sprob))
so_bravo_data = (so_bravo, "SO BRAVO", so_bravo.next_sample_size(sprob))
eor_bravo_data = (eor_bravo, "EoR BRAVO", eor_bravo.next_sample_size(sprob))

data = (minerva2_data, minerva_data, so_bravo_data, eor_bravo_data)

# Print results
for audit, name, first_round_size in data:
    print("First round size {} for {}".format(first_round_size, name))

First round size 105 for Minerva 2.0
First round size 105 for Minerva
First round size 148 for SO BRAVO
First round size 213 for EoR BRAVO


Note that Minerva 2.0 and its predecessor Minerva 1.0 are the exact same in the first round. (So the same first round size for each of them makes sense.)
# First round stopping condition
Now here is a skeleton of the code that will be used for computing the risk after the first round is drawn.

In [63]:
# First round sample from csv
filename = 'test_sample_.csv' #TODO change to the actual sample found during the audit!
df = pd.read_csv(filename)
df_array = df.to_numpy()
sample = {
    'Approve': sum(df_array[:,1]),
    'Reject': sum(df_array[:,2]),
    'Approve_so': df_array[:,1],
    'Reject_so': df_array[:,2],
}
print(sample)

{'Approve': 65, 'Reject': 40, 'Approve_so': array([1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0]), 'Reject_so': array([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1])}


In [64]:
# Test the stopping condition for this sample in each of the audits 
minerva2_data = [minerva2, "Minerva 2.0", minerva2.execute_round(first_round_size, sample)]
minerva2_data.append(minerva2.pvalue_schedule[-1])
minerva_data = [minerva, "Minerva", minerva.execute_round(first_round_size, sample)]
minerva_data.append(minerva.pvalue_schedule[-1])
so_bravo_data = [so_bravo, "SO BRAVO", so_bravo.execute_round(first_round_size, sample)]
so_bravo_data.append(so_bravo.pvalue_schedule[-1])
eor_bravo_data = [eor_bravo, "EoR BRAVO", eor_bravo.execute_round(first_round_size, sample)]
eor_bravo_data.append(eor_bravo.pvalue_schedule[-1])

data = (minerva2_data, minerva_data, so_bravo_data, eor_bravo_data)

# Print results
for audit, name, stopped, risk in data:
    if stopped:
        print("For {}, the audit stopped with risk {}".format(name, risk))
    else:
        print("For {}, the audit did not stop. The risk was {}".format(name, risk))
    

For Minerva 2.0, the audit stopped with risk 0.015103083942572098
For Minerva, the audit stopped with risk 0.015103083942572098
For SO BRAVO, the audit stopped with risk 0.08993324014381973
For EoR BRAVO, the audit stopped with risk 0.05051740640571271


# Second round size
If we are unlucky, we may have to escalate to another round. 
If so, here is code we could run to determine the second round size.

In [65]:
minerva2_data = (minerva2, "Minerva 2.0", minerva2.next_sample_size(sprob))
minerva_data = (minerva, "Minerva", minerva.next_sample_size(sprob))
so_bravo_data = (so_bravo, "SO BRAVO", so_bravo.next_sample_size(sprob))
eor_bravo_data = (eor_bravo, "EoR BRAVO", eor_bravo.next_sample_size(sprob))

data = (minerva2_data, minerva_data, so_bravo_data, eor_bravo_data)

# Print results
for audit, name, first_round_size in data:
    print("First round size {} for {}".format(first_round_size, name))

First round size 213 for Minerva 2.0
First round size 213 for Minerva
First round size 213 for SO BRAVO
First round size 213 for EoR BRAVO


To compute further stopping conditions and round sizes, it suffices to copy the cells from prior rounds.